In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))

In [2]:
from utils import get_cuda_info

get_cuda_info()

PyTorch version: 2.5.1+cu118
**********
_CUDA version: 
CUDA version:
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Wed_Oct_30_01:18:48_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.6, V12.6.85
Build cuda_12.6.r12.6/compiler.35059454_0

**********
CUDNN version: 90100
Available GPU devices: 1
Device Name: NVIDIA GeForce RTX 4070 Ti SUPER


## Zdobycie danych

In [3]:
from utils import load_data, convert_landmarks_to_eye_movements

all_data, all_labels = load_data('miami_deception')

In [4]:
converted_data, blinks = convert_landmarks_to_eye_movements(all_data)

## Preprocessing danych

In [5]:
from utils import preprocess_data

X_train, X_val, X_test, y_train, y_val, y_test = preprocess_data(converted_data, all_labels, binarize_labels=False)

In [6]:
print(X_train.shape)
print(y_train.shape)

torch.Size([224, 1679, 2])
torch.Size([224])


In [7]:
from utils import get_class_distribution

get_class_distribution(all_labels)

===> Class distribution <===
0: 160
1: 160
=============><=============


# MODEL TORCH

## Zbudowanie modelu ekstrakcji cech

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [9]:
class LieClassifier(nn.Module):
    def __init__(self):
        super(LieClassifier, self).__init__()
        
        # Spatial feature extraction (changed in_channels to 2)
        self.conv1 = nn.Conv1d(in_channels=2, out_channels=32, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
        
        # Temporal feature extraction (need to calculate the conv_output_size dynamically)
        self.lstm_hidden_size = 128  # Hidden size for LSTM
        self.lstm_bidirectional_factor = 2  # For bidirectional LSTM
        
        # We need to calculate the size after the convolutions and pooling
        self.conv_output_size = self.calculate_conv_output_size(2)  # 2 input channels (x, y)

        # Temporal feature extraction
        self.lstm = nn.LSTM(input_size=self.conv_output_size, hidden_size=self.lstm_hidden_size,
                            batch_first=True, bidirectional=True)
        
        # Classification head
        self.attn = nn.Linear(self.lstm_hidden_size * self.lstm_bidirectional_factor, 1)
        self.fc1 = nn.Linear(self.lstm_hidden_size * self.lstm_bidirectional_factor, 64)  # 128*2 for bidirectional
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 1)
        
        self._init_weights()

    def _init_weights(self):
        for name, param in self.named_parameters():
            if 'weight' in name:
                nn.init.xavier_normal_(param)
            elif 'bias' in name:
                nn.init.constant_(param, 0.1)

    def calculate_conv_output_size(self, input_size):
        # Calculate the size of the output after convolution and pooling layers
        # Assuming two pooling layers with stride=2 (each halving the length)
        output_size = input_size
        for _ in range(2):  # For two pooling layers
            output_size = (output_size - 1) // 2 + 1  # Pooling halving the size
        return 64 * output_size  # 64 channels after second conv layer

    def forward(self, x):
        # x shape: (batch_size, frames, 2)
        batch_size, frames, _ = x.shape
        
        # Reshape for Conv1D: (batch*frames, 2, 1)
        x = x.view(-1, 2, 1)
        
        # Spatial features
        x = F.relu(self.conv1(x))
        
        # Apply pooling only if the sequence is large enough
        if x.size(2) > 1:
            x = self.pool1(x)
        
        x = F.relu(self.conv2(x))
        
        # Apply second pooling if the sequence is large enough
        if x.size(2) > 1:
            x = self.pool2(x)
        
        # Prepare for LSTM
        x = x.view(batch_size, frames, -1)
        
        # Temporal features
        x, _ = self.lstm(x)
        attn_weights = torch.softmax(self.attn(x), dim=1)
        x = (x * attn_weights).sum(dim=1)
        
        # Classification head (remove sigmoid!)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)  # Raw logits


In [10]:
from torch.optim import Adam

model = LieClassifier()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
pos_weight = torch.tensor([(len(y_train) - y_train.sum()) / y_train.sum()]).to(device)
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = Adam(model.parameters(), lr=1e-4)

## Trening modelu

In [11]:
from torch.utils.tensorboard import SummaryWriter

RUNS_FOLDER_PATH = os.path.abspath('runs')
writer_path = os.path.join('runs', 'torch_lstm', 'eye_movement')
writer = SummaryWriter(writer_path)

In [12]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [13]:
from utils.model_functions import train_torch_model_binary

train_torch_model_binary(model, criterion, optimizer, X_train, y_train, X_val, y_val, writer=writer, batch_size=32, unbalanced=True, show_prediction_stats=False)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                     TRAINING                                         VALIDATION                    
----------------------------------------------------------------------------------------------------
Loss        : 4.8063                                    Loss        : 1.3013
Accuracy    : 0.4196                                    Accuracy    : 0.6458
Precision   : 0.4151                                    Precision   : 0.3229
Recall      : 0.4187                                    Recall      : 0.5000
F1 Score    : 0.4129                                    F1 Score    : 0.3924
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                   

## Ewaluacja modelu

In [14]:
from utils.model_functions import eval_torch_model_binary

eval_torch_model_binary(model, criterion, X_test, y_test)


                                          EPOCH STATISTICS                                          
Epoch       : 1
----------------------------------------------------------------------------------------------------
                                             VALIDATION                                             
----------------------------------------------------------------------------------------------------
Loss        : 1.3143
Accuracy    : 0.4792
Precision   : 0.2396
Recall      : 0.5000
F1 Score    : 0.3239
----------------------------------------------------------------------------------------------------
                                          VALIDATION EXTRA                                          
TP Rate     : 0.0000                                    FP Rate     : 0.0000



# TODYNET

### Przygotowanie danych

In [15]:
X_train_np = X_train.numpy()
X_val_np = X_val.numpy()
X_test_np = X_test.numpy()
y_train_np = y_train.numpy()
y_val_np = y_val.numpy()
y_test_np = y_test.numpy()

In [16]:
TodyNet_DATA_PATH = os.path.join("..", "..", "src", "external", "TodyNet", "data", "UCR", "MIAMI_DECEPTION_EYE_MOVEMENT")

os.makedirs(TodyNet_DATA_PATH, exist_ok=True)

In [17]:
X_train_tensor = torch.tensor(X_train_np, dtype=torch.float32).unsqueeze(1)  # adding channel dimension
X_val_tensor = torch.tensor(X_val_np, dtype=torch.float32).unsqueeze(1)
X_test_tensor = torch.tensor(X_test_np, dtype=torch.float32).unsqueeze(1)

# Save the data in PyTorch (.pt) format
torch.save(X_train_tensor, os.path.join(TodyNet_DATA_PATH, 'X_train.pt'))
torch.save(X_val_tensor, os.path.join(TodyNet_DATA_PATH, 'X_valid.pt'))
torch.save(X_test_tensor, os.path.join(TodyNet_DATA_PATH, 'X.pt'))

# Save the labels in PyTorch (.pt) format
torch.save(y_train, os.path.join(TodyNet_DATA_PATH, 'y_train.pt'))
torch.save(y_val, os.path.join(TodyNet_DATA_PATH, 'y_valid.pt'))
torch.save(y_test, os.path.join(TodyNet_DATA_PATH, 'y.pt'))

In [18]:
X_train_tensor.shape

torch.Size([224, 1, 1679, 2])

### Trening modelu [pool_ratio 0.8, ponieważ rozmiar danych jest zbyt duży na 0.2]

In [ ]:
# cd .\src\external\TodyNet\src\ & python train.py --dataset='MIAMI_DECEPTION_EYE_MOVEMENT' --num_layers 1 --in_dim 16 --hidden_dim 16 --out_dim 16 --pool_ratio 0.0 --kern_size "3" --groups 1